<a href="https://colab.research.google.com/github/adarsh-meher/MachineHack---Predict-News-Category/blob/master/MH_Predict_News_category.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install fasttext
!pip install torchtext
!pip install transformers

     |████████████████████████████████| 61kB 1.8MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2385714 sha256=c26dd3d112037c1e5aebd1c0e137c7ba395904009a9b14ef50492efdb3702308
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext
     |████████████████████████████████| 573kB 2.8MB/s 
     |████████████████████████████████| 3.7MB 42.2MB/s 
     |████████████████████████████████| 1.0MB 41.2MB/s 
     |████████████████████████████████| 890kB 40.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=a9e51578be9b6e26cada3ad40b306dbeeec238c9d12ce96efef9387f0f8c4eb3
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [4]:
!git clone https://github.com/adarsh-meher/MachineHack---Predict-News-Category.git
import os
os.chdir('/content/MachineHack---Predict-News-Category/')

Cloning into 'MachineHack---Predict-News-Category'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 21 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [5]:
import torch
import pandas as pd
import numpy as np
import scipy as sc
import seaborn as sb
import matplotlib.pyplot as plt
import time
import nltk
import gensim as G
import spacy as S
import fasttext as ft

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
news_data = pd.ExcelFile('Data_Train.xlsx').parse('Sheet1')

In [8]:
news_data.head()

,STORY,SECTION
0,But the most painful was the huge reversal in ...,3
1,How formidable is the opposition alliance amon...,0
2,Most Asian currencies were trading lower today...,3
3,"If you want to answer any question, click on ‘...",1
4,"In global markets, gold prices edged up today ...",3


In [9]:
#### Counts of news category
print(news_data['SECTION'].value_counts()/news_data.shape[0])

'''
Politics: 0
Technology: 1
Entertainment: 2
Business: 3
'''

### no imbalance in categories

target_to_name_map = {0:'Politics',1:"Technology",2:"Entertainment",3:"Business"}

1    0.363398
2    0.252229
0    0.221028
3    0.163346
Name: SECTION, dtype: float64


In [0]:
#### Creating a test set assuming similar proportion of categories in target
news_train,news_test,target_train,target_test = train_test_split(news_data.drop(['SECTION'],axis = 1),news_data['SECTION'],test_size = 0.2,stratify = news_data['SECTION'])
news_train.index,target_train.index = range(news_train.shape[0]),range(news_train.shape[0])
news_test.index,target_test.index = range(news_test.shape[0]),range(news_test.shape[0])

## ***Analyzing news_train texts***

In [0]:
df = pd.concat([news_train.rename(columns = {'STORY' : 'text'}),pd.Series(target_train,name = 'target')],axis = 1)

In [12]:
df.head()

,text,target
0,The first season of the show will represent Th...,2
1,Fusion skills give companies and people a fram...,1
2,The president-elect of Ukraine does not have a...,2
3,Singer Rob Thomas says he owes it to guitar le...,2
4,Lau did not share any more details of the plan...,1


In [13]:
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,LancasterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download("stopwords")
nltk.download("punkt")
stop_words = set(stopwords.words("english"))
ps = PorterStemmer()
ls = LancasterStemmer()
wns = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
##### Freq Dist of all words for all categories except stopwords
tokens = [ y for x in df['text'].values for y in word_tokenize(x) if y.lower() not in stop_words and y.isalpha()]
tokens_fdist = FreqDist(tokens)

In [15]:
#### Freq distribution of words for each category
token_cat = {}
token_cat_fdist = {}
for i in df['target'].unique():
  print(target_to_name_map[i])
  token_cat[target_to_name_map[i]] = [ y for x in df[df['target'] == i]['text'].values for y in word_tokenize(x) if y.lower() not in stop_words and y.isalpha() ]
  token_cat_fdist[target_to_name_map[i]] = FreqDist(token_cat[target_to_name_map[i]]) 

Entertainment
Technology
Business
Politics


In [0]:
#### Dataframe to show for each category the top n words
n = 100
top_words_each_cat = pd.DataFrame({"Technology" : [i for i,v in token_cat_fdist["Technology"].most_common(n)],
                                      "Politics" : [i for i,v in token_cat_fdist["Politics"].most_common(n)],
                                      "Entertainment" : [i for i,v in token_cat_fdist["Entertainment"].most_common(n)],
                                      "Business" : [i for i,v in token_cat_fdist["Business"].most_common(n)]},index=range(n))

In [17]:
top_words_each_cat

,Technology,Politics,Entertainment,Business
0,said,BJP,film,said
1,also,Congress,said,year
2,Apple,said,also,market
3,India,Modi,like,India
4,new,party,actor,crore
...,...,...,...,...
95,system,manifesto,Game,Reliance
96,products,Telangana,every,Tuesday
97,many,security,among,increase
98,come,Bengal,MCU,time


In [0]:
#### Extract set of words that appear both of any of the two categories
from itertools import combinations
common_words = []
for i in combinations(target_to_name_map.values(),2):
  [ common_words.append(x) for x in set(top_words_each_cat[i[0]]).intersection(set(top_words_each_cat[i[1]])) ]

common_words = set(list(common_words))

In [0]:
#### Dataframe to show for each category the bottom n words
n = 100
least_words_each_cat = pd.DataFrame({"Technology" : [i for i,v in token_cat_fdist["Technology"].most_common()[-n:]],
                                      "Politics" : [i for i,v in token_cat_fdist["Politics"].most_common()[-n:]],
                                      "Entertainment" : [i for i,v in token_cat_fdist["Entertainment"].most_common()[-n:]],
                                      "Business" : [i for i,v in token_cat_fdist["Business"].most_common()[-n:]]},index=range(n))

In [20]:
least_words_each_cat

,Technology,Politics,Entertainment,Business
0,multiplexes,meters,Cardellini,Marcella
1,cinema,suspect,Christou,Chow
2,discoverability,CSR,Kinchen,shook
3,cataloguing,deserves,Raymond,chugging
4,copyrighting,Seniors,muddied,Pavlik
...,...,...,...,...
95,Slate,eldest,deserved,tourism
96,equip,Sharada,recognised,discrepancy
97,Lagoon,Venkata,hurtful,happening
98,Viola,Hazare,quickest,Reflecting


# **Cleaning text data**

In [0]:
from html.parser import HTMLParser
import re
import string

In [0]:
contrac = {'s' : 'is','t':'not','ve' : 'have','d' : 'had', '':'a'}

def clean_text(x):
  ###### Remove HTML characters
  x = HTMLParser().unescape(x)

  ##### Treating apostrophes/contraction words
  #x = ' '.join([ w.split('\'')[0].lower()[:-1]+' '+contrac[w.split('\'')[1].lower()] if '\'' in w else w for sent in sent_tokenize(x) for w in sent.split()  ])
  word_list = []
  for sent in sent_tokenize(x):
    for w in sent.split():
      if '\'' in w:
        w1 = w.split('\'')[0].lower()[:-1]
        try:
          c1 = w.split('\'')[1]/lower()
          w_new = w1 + ' '+ contrac[w2]
          word_list.append(w_new)
        except:
          w_new = w1
          word_list.append(w_new)
      else:
        word_list.append(w)
    
  x = ' '.join(word_list) 

  ##### Removing stopwords and stemming words to its base form
  x = ' '.join([ ps.stem(w) for w in word_tokenize(x) if w not in stop_words])

  ##### Remove punctuations
  table = str.maketrans('', '', string.punctuation)
  x = ' '.join([w.translate(table) for w in word_tokenize(x) ])

  ##### Replace digits with 'numeral' string to denote presence of number. This can be used in business category as presence of number in that category is certainly possible
  x = ' '.join([ 'numeral' if w.isdigit() else w for w in word_tokenize(x)])

  ##### Remove presence of url/https links
  url_pattern = r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
  x = re.sub(url_pattern, ' ', x)

  ##### Remove nonascii characters
  x = ''.join([chr for chr in x if ord(chr) < 128])

  ##### strip and leading/training spaces and convert all words to lowercase
  x = x.strip().lower()

  return x

  

In [23]:
news_train['clean_text'] = news_train['STORY'].apply(lambda x : clean_text(x))
news_test['clean_text'] = news_test['STORY'].apply(lambda x : clean_text(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  """


In [0]:
pd.concat([target_train,news_train.drop(['STORY'],axis = 1)],axis = 1).to_csv('News_Train.csv',index = False)
pd.concat([target_test,news_test.drop(['STORY'],axis = 1)],axis = 1).to_csv('News_Test.csv',index = False)

### **`Classification using Torch`**

Simple LSTM Baseline model using TorchText

In [0]:
import torch 
from torchtext.data import Field,BucketIterator,TabularDataset,LabelField
import torch.nn as nn

In [0]:
text_field = Field(tokenize = word_tokenize,lower = True)
label_field = LabelField()
datafields = [('SECTION',label_field),('clean_text',text_field)]

In [0]:
train = TabularDataset(path = os.path.join(os.getcwd(),'News_Train.csv'),format = "csv",skip_header=True,fields=datafields)
test = TabularDataset(path = os.path.join(os.getcwd(),'News_Test.csv'),format = "csv",skip_header=True,fields=datafields)

In [0]:
text_field.build_vocab(train,max_size = 5000)
label_field.build_vocab(train)

In [0]:
train_iter = BucketIterator(train,batch_size=64,sort_key=lambda x: len(x.clean_text),sort_within_batch=False)
test_iter = BucketIterator(test,batch_size=64,sort_key=lambda x: len(x.clean_text),sort_within_batch=False)

In [0]:
##### Defining own RNN module
class RNN(nn.Module):
  def __init__(self,input_dim,embed_dim,hidden_dim,output_dim):
    super(RNN,self).__init__()

    self.hidden_dim = hidden_dim
    self.embed = nn.Embedding(input_dim,embed_dim)
    self.i2h = nn.Linear(embed_dim + hidden_dim,hidden_dim)
    self.i2o = nn.Linear(embed_dim + hidden_dim,output_dim)
    self.softmax = nn.LogSoftmax(dim = 1)

  
  def forward(self,text,hidden):
    embeds = self.embed(text)
    print(embeds.shape)
    embed_hid_concat = torch.cat((embeds,hidden),1)
    hidden = self.i2h(embed_hid_concat)
    output,hidden = self.i2o(embed_hid_concat)
    softmax_output = self.softmax(output)

    return softmax_output,hidden

  def initHidden(self):
    return torch.zeros(1,self.hidden_dim)


In [0]:
input_dim = len(text_field.vocab)
embed_dim = 200
hidden_dim = 300
output_dim = len(np.unique(target_train))

In [0]:
criterion = nn.NLLLoss()
learning_rate = 0.001
model = RNN(input_dim,embed_dim,hidden_dim,output_dim)
init_hidden = model.initHidden()

from torch.optim import Adam
optimizer = Adam(model.parameters(),lr = learning_rate)

In [0]:

def train(model,iterator,optimizer,creterion):

  epoch_loss = 0
  epoch_acc = 0
  model.train()

  for batch in iterator:
    hidden = init_hidden  
    print(hidden.shape)
    optimizer.zero_grad()

    preds,hidden = model(batch.clean_text,hidden)
    print(preds.shape)
    loss = criterion(batch.SECTION,preds)
    loss.backward()
    optimizer.step()


In [94]:
preds,hidden,labels = train(model,train_iter,optimizer,criterion)

torch.Size([1, 300])
torch.Size([283, 64, 200])


RuntimeError: ignored

In [0]:
t1 = torch.zeros(283,1,200)
t2 = torch.ones(283,64,200)

In [123]:
t1.shape

torch.Size([283, 1, 200])

In [124]:
t2.shape

torch.Size([283, 64, 200])

In [125]:
t1.squeeze().shape

torch.Size([283, 200])